# 02. 전이 학습 (Transfer Learning) 실습: 네이버 영화 리뷰 감성 분석
### 과목: NLP Transfer Learning
---
**학습 목표**
1. **전이 학습(Transfer Learning)**의 개념을 이해하고, 왜 강력한지 깨닫는다.
2. 사전 학습된 **BERT (klue/bert-base)** 모델을 불러온다.
3. **NSMC (네이버 영화 리뷰)** 데이터셋으로 BERT를 미세 조정(Fine-tuning)하여 감성 분석 모델을 만든다.

### 🍎 쉬운 비유: "천재 의대생의 피부과 개원기"
- **사전 학습 (Pre-training):** 똑똑한 의대생(BERT)이 6년 동안 수만 가지 의학 서적(위키피디아, 뉴스 등)을 읽으며 '일반적인 의학 지식'을 쌓습니다.
- **전이 학습 (Transfer Learning):** 이 의대생이 '피부과' 전문의가 되기로 결심합니다.
- **미세 조정 (Fine-tuning):** 이제 의학 서적은 그만 보고, '피부과 전공 서적(NSMC 데이터)'만 집중적으로 봅니다. 이미 기초가 튼튼해서 금방 전문가가 됩니다.
- **결과:** 처음부터 의학을 배우는 것보다(처음부터 학습), 의대생 데려와서 피부과만 가르치는 게(전이 학습) 훨씬 빠르고 성능이 좋습니다.

### 1. 데이터 준비하기 (NSMC)
네이버 영화 긍정/부정 리뷰 데이터를 다운로드합니다.

In [ ]:
import pandas as pd
from tensorflow.keras.utils import get_file

# 1. 인터넷에서 데이터 파일 다운로드 (케라스 유틸리티 사용)
# train: 학습용 데이터, test: 평가용 데이터
train_file = get_file('ratings_train.txt', origin='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_file = get_file('ratings_test.txt', origin='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

# 2. 판다스로 읽어오기 (구분자는 탭문자 '\t')
ratings_train_df = pd.read_csv(train_file, sep='\t')
ratings_test_df = pd.read_csv(test_file, sep='\t')

# 데이터 모양 확인
print("학습 데이터 개수:", len(ratings_train_df))
print("테스트 데이터 개수:", len(ratings_test_df))
ratings_train_df.head() # 상위 5개 출력

In [ ]:
# 3. 결측치 제거 (빈 줄이 있을 수 있음)
ratings_train_df = ratings_train_df.dropna(how='any')
ratings_test_df = ratings_test_df.dropna(how='any')

print("결측치 제거 후 학습 데이터:", len(ratings_train_df))

In [ ]:
# [실습 편의를 위해 데이터 일부만 사용]
# 전체 데이터를 다 쓰면 시간이 너무 오래 걸리니, 학습용 1.5만 개, 테스트용 5천 개만 뽑아서 씁니다.
ratings_train_df = ratings_train_df.sample(n=15000, random_state=0)
ratings_test_df = ratings_test_df.sample(n=5000, random_state=0)

# 데이터(X)와 라벨(y) 분리
X_train = ratings_train_df['document'].values.tolist()
y_train = ratings_train_df['label'].values.tolist()

X_test = ratings_test_df['document'].values.tolist()
y_test = ratings_test_df['label'].values.tolist()

print("샘플링 후:", len(X_train), len(X_test))

### 2. 토크나이저 & 모델 불러오기
한국어 처리에 특화된 **KLUE-BERT** 모델을 사용합니다.

In [ ]:
from transformers import AutoTokenizer, TFBertForSequenceClassification

MODEL_NAME = 'klue/bert-base' # 한국어 BERT 모델

# 1. 토크나이저 로드 (한국어 단어장 + 쪼개는 규칙)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 2. 모델 로드 (머리)
# num_labels=2: 긍정(1), 부정(0) 2개로 분류하니까 2개.
# from_pt=True: 파이토치로 학습된 가중치를 텐서플로우로 변환해서 가져온다는 뜻
model = TFBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2, from_pt=True)

In [ ]:
# 3. 데이터 토큰화 (글자 -> 숫자)
# padding=True: 문장 길이를 맞춰줌 (짧은 문장은 뒤에 0을 채움)
# truncation=True: 너무 긴 문장은 자름 (BERT는 최대 512자까지만 가능)

X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, return_tensors='tf')
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, return_tensors='tf')

# 결과 확인: 첫 번째 문장이 어떻게 바뀌었나?
print("원본:", X_train[0])
print("숫자(인코딩):", X_train_tokenized['input_ids'][0])

### 3. 고성능 데이터 파이프라인 만들기 (tf.data)
대량의 데이터를 효율적으로 학습시키기 위해 텐서플로우의 `Dataset` 기능을 사용합니다.

In [ ]:
import tensorflow as tf

# 1. 데이터셋 생성 (입력값 dict + 라벨)
train_ds = tf.data.Dataset.from_tensor_slices((dict(X_train_tokenized), y_train))
test_ds = tf.data.Dataset.from_tensor_slices((dict(X_test_tokenized), y_test))

# 2. 셔플(섞기) & 배치(묶기) & 프리페치(미리로딩)
# prefetch: 학습하는 동안 다음 배치를 미리 메모리에 올려놔서 끊김 없이 학습하게 함
train_dataset = train_ds.shuffle(10000).batch(64).prefetch(tf.data.AUTOTUNE)
test_dataset = test_ds.batch(64).prefetch(tf.data.AUTOTUNE)

print("데이터 파이프라인 준비 완료!")

### 4. 학습 시작 (Fine-tuning)
이제 의대생(BERT)에게 피부과 전공 서적(영화 리뷰)을 가르칠 시간입니다.

In [ ]:
from transformers import create_optimizer

# 1. 최적화 도구(Optimizer) 설정
# Warmup: 처음엔 천천히 배우다가(전체 스텝의 10%) 점점 속도를 냄. 학습 안정성을 위해 중요!
num_train_steps = len(train_dataset) * 3  # Epochs = 3
num_warmup_steps = int(num_train_steps * 0.1)

optimizer, _ = create_optimizer(
    init_lr=5e-5,            # 학습률 (너무 크면 기존 지식을 까먹음)
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    weight_decay_rate=0.01
)

# 2. 모델 컴파일 (학습 준비)
model.compile(
    optimizer=optimizer,
    loss=model.hf_compute_loss, # HuggingFace 모델 전용 손실 함수
    metrics=['accuracy']        # 정확도 측정
)

# 3. 실제 학습 진행 (fit)
# 데이터를 넣고 3번 반복(Epochs=3)해서 공부시킵니다.
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=3
)

### 5. 실전 테스트 (Inference)
학습된 모델이 진짜로 리뷰를 잘 분석하는지 확인해봅시다.

In [ ]:
from transformers import TextClassificationPipeline

# 라벨 이름 붙여주기
model.config.id2label = {0: "부정", 1: "긍정"}
model.config.label2id = {"부정": 0, "긍정": 1}

# 사용하기 편하게 파이프라인으로 만들기
sentiment_classifier = TextClassificationPipeline(
    tokenizer=tokenizer,
    model=model,
    framework='tf',
    top_k=1  # 가장 확률 높은 1개만 출력
)

In [ ]:
# 직접 문장을 써서 테스트해보세요!
reviews = [
    "이 영화 진짜 인생 영화다... 눈물 콧물 쏙 뺐음 ㅠㅠ",
    "돈 아까워 죽는 줄 알았다. 도대체 뭘 말하고 싶은 건지 모르겠음.",
    "배우들 연기는 좋은데 스토리가 좀 산으로 가네."
]

for review in reviews:
    result = sentiment_classifier(review)
    print(f"리뷰: {review}\n결과: {result}\n")

---

### (선택) 내 모델 저장하고 공유하기
`push_to_hub`를 사용하면 Hugging Face Hub에 내 모델을 업로드해서 언제든 다시 꺼내 쓸 수 있습니다.
*(이 단계는 Hugging Face 계정과 토큰이 필요합니다)*

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login() # 여기서 로그인

# REPO_NAME = 'my-first-bert-nsmc' 
# model.push_to_hub(REPO_NAME)
# tokenizer.push_to_hub(REPO_NAME)